<br>
<img src="data/instagram_logo.png" alt="Logo de Instagram" style="width:450px"/>

# **Instagram NLP Posts Classifier**<br>

### 👨‍💻 Jorge Gómez Galván
* LinkedIn: [linkedin.com/in/jorgeggalvan/](https://www.linkedin.com/in/jorgeggalvan/)
* E-mail: ggalvanjorge@gmail.com
---

## **Clasificación multiclase mediante Machine Learning**

In [1]:
# Importación de librerías
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Librerías de Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler

In [2]:
# Lectura del dataset
df_posts = pd.read_csv('./data/instagram_posts_lemmatized.csv')
df_posts = df_posts.drop('Unnamed: 0', axis='columns') # Eliminación de columnas irrelevantes

df_posts.head()

,category,text_lemmatized
0,fashion,forcing bridesmaid join # ootd # mirrormoment ...
1,pet,catman : get anything done ? : welcome world 😀...
2,fashion,"guy please check @ awa_khiwe , really talented..."
3,travel,"world feel like 's falling apart , 's importan..."
4,fashion,last night 🤫🤫🤫 .. # pink # club # nightout # d...


### **1 - Clasificación de texto**

#### 1.1 - Matriz de frecuencias

In [3]:
# Matriz TF-IDF
tv = TfidfVectorizer() # Inicialización del vectorizador TF-IDF
tv_matrix = tv.fit_transform(df_posts['text_lemmatized']) # Aplicación del vectorizador TF-IDF a la columna 'text_lemmatized'

<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
La matriz TF-IDF (Term Frequency-Inverse Document Frequency) es una representación numérica que mide la importancia relativa de las palabras en un conjunto de textos. Cada fila representa un texto y cada columna representa una palabra lematizada, con los valores numéricos correspondientes a su TF-IDF.
<br><br>
Este algoritmo se utiliza para transformar textos en representaciones numéricas que pueden ser utilizadas en modelos de Machine Learning para diferentes objetivos como clasificación de texto.
</div>

#### 1.2 - Modelo de clasificación

In [4]:
# Variable dependiente
y = df_posts['category']

# Variables independientes
x = tv_matrix

# Conjunto de datos de entrenamiento y de prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

In [5]:
# Modelo SVC (Support Vector Classification)
svc = SVC()
svc_model = svc.fit(x_train, y_train)

# Predicción con SVC
y_pred = svc.predict(x_test)

In [6]:
# Métricas del modelo
precision = precision_score(y_test, y_pred, average='weighted').round(3)
recall = recall_score(y_test, y_pred, average='weighted').round(3)
f1 = f1_score(y_test, y_pred, average='weighted').round(3)

print("Métricas del modelo:")
print("- Precision:", precision)
print("- Recall:", recall)
print("- F1-Score:", f1)

Métricas del modelo:
- Precision: 0.645
- Recall: 0.611
- F1-Score: 0.599


<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
El modelo alcanza una precision y un recall global del 64% y del 61%, respectivamente, en la clasificación de las categorías de las publicaciones. 
<br><br>
En otras palabras, el modelo clasifica correctamente alrededor de 6 de cada 10 publicaciones del conjunto de datos de test. Además, de todas las publicaciones que realmente pertenecen a una categoría específica, el modelo logra recuperar el 61% de ellas.
</div>

In [7]:
# Informe de clasificación en test
clf_report_test = classification_report(y_test, y_pred)

print("Informe de clasificación en test:\n\n", clf_report_test)

Informe de clasificación en test:

               precision    recall  f1-score   support

      beauty       0.79      0.38      0.51      7308
      family       0.63      0.43      0.51     23652
     fashion       0.54      0.86      0.66     60244
     fitness       0.69      0.30      0.42      6558
        food       0.78      0.64      0.71     22463
    interior       0.80      0.47      0.60      7217
       other       0.63      0.53      0.58     29454
         pet       0.78      0.44      0.56      3587
      travel       0.70      0.43      0.53     21788

    accuracy                           0.61    182271
   macro avg       0.70      0.50      0.56    182271
weighted avg       0.64      0.61      0.60    182271



<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
Al observar las métricas desglosadas por categoría, se puede apreciar que el rendimiento varía según las clases:
<br><br>
El modelo destaca por clasificar con una <u>precisión alta las publicaciones de las categorías 'interior', 'beauty', 'food' y 'pet'</u>. Sin embargo, presenta <u>dificultades para otras categorías como 'fashion', 'family' y 'other'</u>, que son curiosamente las que cuentan con el mayor número de muestras.
<br><br>
Es evidente que la categoría <u>'fashion' destaca por tener el recall más alto, pero también la precisión más baja</u>, por lo que tiene problemas en la distinción precisa entre el contenido de moda y otras categorías al ser 'fashion' la categoría con mayor cantidad de publicaciones en el dataset. En contraste, <u>'food' exhibe el equilibrio más notable</u> al tener el F1-Score más alto, combinando una alta precisión y recall. Este equilibrio podría estar relacionado con las características distintivas del contenido de comida.
<br><br>   
Además, <u>las clases con un menor volumen de publicaciones</u> ('pet', 'fitness', 'interior' y 'beauty') <u>son, salvo 'fitness', las clases con mayor tasa de presición</u>. Esto se debe a que el modelo, al tener menos datos para aprender de estas categorías, realiza predicciones más seguras y menos propensas a errores, lo que justifica sus valores bajos de recall.
</div>

In [8]:
# Matriz de confusión en test
confusion_matrix_test = confusion_matrix(y_test, y_pred)

print("Matriz de confusión en test:\n\n", confusion_matrix_test)

Matriz de confusión en test:

 [[ 2742   145  3719    15    56     5   536    13    77]
 [   69 10276  9558   125   843   223  1876    86   596]
 [  342  1950 52097   261  1067   208  2665    98  1556]
 [    8   239  3209  1952   206     6   681    20   237]
 [   59   871  4933   111 14406   159  1247    62   615]
 [    9   616  2374     1   187  3417   414    18   181]
 [  208  1359 10283   246   913   120 15559    90   676]
 [    1   227  1382     3    57     9   230  1576   102]
 [   20   743  9421   125   618    98  1400    68  9295]]


<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
La matriz de confusión generada ofrece una perspectiva clara sobre cómo el modelo clasifica las diferentes categorías y dónde se producen las confusiones. 
<br><br> 
En la tercera columna, que corresponde a la categoría 'fashion', se puede observar que la mayor cantidad de errores de clasificación ocurren dentro de esta categoría. Además, la categoría 'other' también contribuye a un número considerable de errores en las predicciones. Estos resultados pueden atribuirse en parte a la amplitud y diversidad de la clase 'other', la cual abarca una gran variedad de temas y cuyas temáticas de contenido no están bien definidas.
</div>

In [9]:
# Matriz de confusión en train
confusion_matrix_train = confusion_matrix(y_train, svc.predict(x_train))

print("Matriz de confusión en train:\n\n", confusion_matrix_train)

Matriz de confusión en train:

 [[ 11278    261   4977     33    100     16    581     25    157]
 [   122  45111   7154    131    855    166   1096     57    521]
 [   198   1117 137192    123    777    124    529     50    615]
 [    26    323   4160   9558    335     11    604     24    267]
 [    94    962   4116    122  45463    153    700     58    659]
 [    14    748   2585      5    247  13086    328     28    201]
 [   154   1061   5806    155    964    108  59637     70    683]
 [     1    202   1696      3     83     12    196   6023     97]
 [    59    884   7743    133    750     90    943     63  39437]]


<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
La matriz de confusión en el conjunto de entrenamiento muestra cómo el modelo clasifica las muestras en diferentes categorías y ayuda a determinar si el modelo está generalizando adecuadamente o si está sobreajustando los datos de entrenamiento.
</div>

### **2 - Clasificación de texto con undersampling**

#### 2.1 - Balanceo de clases

In [10]:
# Número de publicaciones por categoría
category_post = df_posts['category'].value_counts()
category_post = category_post.to_frame(name='n_posts').reset_index().rename(columns={'index':'category'})

category_post

,category,n_posts
0,fashion,200969
1,other,98092
2,family,78865
3,food,74790
4,travel,71890
5,beauty,24736
6,interior,24459
7,fitness,21866
8,pet,11900


In [11]:
# Número mínimo de muestras a retener para cada clase
min_samples = category_post['n_posts'].min()

print("Mínimo de muestras por clase: ", min_samples)

Mínimo de muestras:  11900


In [12]:
# Creación de RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy = {category: min_samples for category in df_posts['category'].unique()}, random_state=42)

# Aplicación de undersampling al dataframe
x_resampled, y_resampled = rus.fit_resample(df_posts['text_lemmatized'].values.reshape(-1, 1), df_posts['category'])

# Creación del dataframe balanceado
df_posts_balanced = pd.DataFrame({'category': y_resampled, 'text_lemmatized': x_resampled.flatten()})

In [13]:
# Tamaño del dataset balanceado
df_posts_balanced.shape

(107100, 2)

<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
El análisis del informe de clasificación muestra disparidades en la precisión y el recall entre categorías que pueden deberse a los desequilibrios existentes en la cantidad de muestras por categoría. Este desbalanceo de clases puede producir un deterioro importante en la efectividad del modelo, impactando en cómo el modelo generaliza y perjudicando a las clases minoritarias.
<br><br>    
Balancear las clases podría ser una buena opción para mitigar este sesgo y permitir un aprendizaje más equilibrado que mejore la precisión general del modelo en todas las categorías.
</div>

#### 2.2 - Matriz de frecuencias

In [14]:
# Matriz TF-IDF
tv_balanced = TfidfVectorizer() # Inicialización del vectorizador TF-IDF
tv_matrix_balanced = tv_balanced.fit_transform(df_posts_balanced['text_lemmatized']) # Aplicación del vectorizador TF-IDF a la columna 'text_lemmatized'

#### 2.3 - Modelo de clasificación

In [15]:
# Variable dependiente
y_balanced = df_posts_balanced['category']

# Variables independientes
x_balanced = tv_matrix_balanced

# Conjunto de datos de entrenamiento y de prueba
x_balanced_train, x_balanced_test, y_balanced_train, y_balanced_test = train_test_split(x_balanced, y_balanced, test_size=0.3, random_state=123)

In [16]:
# Modelo SVC (Support Vector Classification)
svc_balanced = SVC()
svc_model_balanced = svc_balanced.fit(x_balanced_train, y_balanced_train)

# Predicción con SVC
y_balanced_pred = svc_balanced.predict(x_balanced_test)

In [17]:
# Métricas del modelo
precision_balanced = precision_score(y_balanced_test, y_balanced_pred, average='weighted').round(3)
recall_balanced = recall_score(y_balanced_test, y_balanced_pred, average='weighted').round(3)
f1_balanced = f1_score(y_balanced_test, y_balanced_pred, average='weighted').round(3)

print("Métricas del modelo:")
print("- Precision:", precision_balanced)
print("- Recall:", recall_balanced)
print("- F1-Score:", f1_balanced)

Métricas del modelo:
- Precision: 0.566
- Recall: 0.54
- F1-Score: 0.548


In [18]:
# Informe de clasificación en test
clf_report_balanced_test = classification_report(y_balanced_test, y_balanced_pred)

print("Informe de clasificación en test:\n\n", clf_report_balanced_test)

Informe de clasificación en test:

               precision    recall  f1-score   support

      beauty       0.68      0.56      0.61      3627
      family       0.44      0.45      0.45      3598
     fashion       0.40      0.39      0.39      3649
     fitness       0.57      0.52      0.54      3477
        food       0.70      0.62      0.65      3556
    interior       0.73      0.62      0.67      3466
       other       0.34      0.53      0.41      3554
         pet       0.77      0.66      0.71      3589
      travel       0.48      0.52      0.50      3614

    accuracy                           0.54     32130
   macro avg       0.57      0.54      0.55     32130
weighted avg       0.57      0.54      0.55     32130



<div style="border: 5px solid #FF0069; padding: 20px; font-size: 16px; background-color: rgba(255, 0, 105, 0.2);">
Sin embargo, al examinar el informe de clasificación tras haber balanceado las clases, se observa que no solo la precisión no mejora, sino que empeora. No obstante, sí que hay una mejora en el recall de todas las categorías excepto en 'fashion'.
</div>

In [19]:
# Matriz de confusión en test
confusion_matrix_balanced_test = confusion_matrix(y_balanced_test, y_balanced_pred)

print("Matriz de confusión en test:\n\n", confusion_matrix_balanced_test)

Matriz de confusión en test:

 [[2014  187  430  180   43   39  521   66  147]
 [ 129 1618  333  205  176  169  578  116  274]
 [ 281  317 1424  244  107  143  580  102  451]
 [ 110  224  288 1809  146   32  523   76  269]
 [  75  204  155  122 2187  127  387   56  243]
 [  61  322  211   46  109 2143  275   72  227]
 [ 178  324  275  263  154  112 1901   96  251]
 [  35  220  154   85   71   79  376 2356  213]
 [  76  240  325  210  148  108  509  103 1895]]


In [20]:
# Matriz de confusión en train
confusion_matrix_balanced_train = confusion_matrix(y_balanced_train, svc_balanced.predict(x_balanced_train))

print("Matriz de confusión en train:\n\n", confusion_matrix_balanced_train)

Matriz de confusión en train:

 [[7695   97  120   78   44   22   79   40   98]
 [  60 7668  105   71   97   77   64   52  108]
 [ 113  143 7503   81   71   65   73   45  157]
 [  52  102   85 7855  112   26   57   38   96]
 [  60  147   79   90 7680   67   59   42  120]
 [  37  142   71   24   68 7939   42   40   71]
 [  75  124   60  113  108   60 7636   64  106]
 [  20   60   46   35   37   34   42 7983   54]
 [  57  114  116   96   92   55   63   73 7620]]
